<font color='blue'>Here I will document my method for cleaning my data files, which were taken from Kaggle's dataset “Comments on articles published in the New York Times” (https://www.kaggle.com/aashita/nyt-comments).

This code is broken into two sections - one cleans the data files with articles and one cleans the data files with comments. Here I test my cleaning using only a single file, but by concatenating data files together, I can run this code on the entire dataset.</font> 

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
sent_token = nltk.sent_tokenize
import csv  
from nltk import sent_tokenize, word_tokenize, pos_tag
import re
from sklearn.feature_extraction.text import CountVectorizer
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allArticles.csv")
train = pd.read_csv("/root/Springboard/Data/ArticlesApril2017.csv")

train.head(5)

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


<font color='blue'>Initial goals: 

-Make sure the contents of each field are the correct type and have no missing data (i.e. scrub the 'NaN' from the 'abstract' field)

-Make sure that the data comes properly tokenized

-Convert all words to lowercase (to avoid confusion between uppercase and lowercase versions of the same word)

Several of these data columns (articleID, articleWordCount, multimedia, printPage) contain only integers or single lowercase words.</font> 

In [2]:
#for column in train:
    #print(train[column].get_dtype_counts())

<font color='blue'>From the above code, the only integer columns are 2, 7 and 9. The rest are string columns and need to be converted to lowercase. </font> 

In [3]:
sampleSize = 4
train = pd.read_csv("/root/Springboard/Data/ArticlesApril2017.csv", header=0, nrows=sampleSize)
#train.head(5)

wn = nltk.WordNetLemmatizer()

nonstrings = [2, 7, 9]


train= train.astype(str)
train.fillna(0)

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in nonstrings:
            doc[column] = doc[column].astype(str)
            continue
        doc[column] = doc[column].str.replace('[^\w\s]','')
        doc[column] = doc[column].str.lower()
        #doc[column] = doc[column].str.strip()
        doc[column] = doc[column].replace(np.nan, '', regex=True)
        doc[column].apply(nltk.word_tokenize)
        doc[column].apply(lemmatize_text)
        doc[column] = [token for token in doc[column] if token not in stop_words]
    return doc

def lemmatize_text(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

#For testing, but this functionality has been compressed into clean_articles
def norm_articles(doc):
    clean_articles(doc)
    doc3 = []
    for index, row in doc.iterrows():
        element = str(row.to_frame().T)
        print(index)
        tokenData = nltk.word_tokenize(element)
        lemma_tokens = lemmatize_text(tokenData)
        #print(tokenData)
        #print("EFFEW")
        filtered_tokens = [token for token in lemma_tokens if token not in stop_words]
        doc2 = ' '.join(filtered_tokens)
        doc3.append(doc2)
    #print(type(doc3))
    return doc3



normalize_corpus = np.vectorize(norm_articles)

clean_art = clean_articles(train)

clean_art.head(5)
#pprint(clean_art2)

#print('\n'.join(clean))


,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,nan,58def1347c459f24986d7c80,716,by stephen hiltner and susan lehman,article,finding an expansive view of a forgotten peop...,photography new york times niger ferguson adam...,3,insider,2,20170401 001541,unknown,one of the largest photo displays in times his...,the new york times,news,httpswwwnytimescom20170331insidernigermigrants...
1,nan,58def3237c459f24986d7c84,823,by gail collins,article,and now the dreaded trump curse,united states politics and government trump do...,3,oped,23,20170401 002358,unknown,meet the gang from under the bus,the new york times,oped,httpswwwnytimescom20170331opinionandnowthedrea...
2,nan,58def9f57c459f24986d7c90,575,by the editorial board,article,venezuelas descent into dictatorship,venezuela politics and government maduro nicolas,3,editorial,22,20170401 005306,unknown,a court ruling annulling the legislatures auth...,the new york times,editorial,httpswwwnytimescom20170331opinionvenezuelasdes...
3,nan,58defd317c459f24986d7c95,1374,by michael powell,article,stain permeates basketball blue blood,basketball college university of north carolin...,3,sports,1,20170401 010652,college basketball,for two decades until 2013 north carolina enga...,the new york times,news,httpswwwnytimescom20170331sportsncaabasketball...


<font color='blue'>Now we want a separate section to clean the comment files. </font> 

In [4]:
#For all data - train = pd.read_csv("/root/Springboard/Data/cleaning/allComments.csv")

sampleSize = 1000
train = pd.read_csv("/root/Springboard/Data/CommentsApril2017.csv", nrows=sampleSize)

train= train.astype(str)
train.fillna(0)
strings = [1, 5, 10, 18, 22, 24, 25, 26, 29, 30, 33]

def lemmatize_text(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

def clean_articles(doc):
    for index, column in enumerate(doc):
        if index in strings:         
            doc[column] = doc[column].str.replace('[^\w\s]','')
            doc[column] = doc[column].str.lower()
            #doc[column] = doc[column].str.strip()
            doc[column] = doc[column].replace(np.nan, '', regex=True)
            doc[column].apply(nltk.word_tokenize)
            doc[column].apply(lemmatize_text)
            doc[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
        else:
            doc[column] = doc[column].astype(str)
            continue
    return doc

normalize_corpus = np.vectorize(norm_articles)

clean_comments = clean_articles(train)
#The second command takes awhile to run
clean_comments.head(5)

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,this project makes me happy to be a 30 year ti...,22022598.0,22022598,<br/>,comment,1491237056.0,1,False,0.0,...,riverside ca,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
1,1491188619,stunning photos and reportage infuriating that...,22017350.0,22017350,nan,comment,1491180489.0,1,False,0.0,...,br,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
2,1491188617,brilliant work from conception to execution iv...,22017334.0,22017334,<br/>,comment,1491179470.0,1,False,0.0,...,raleigh nc,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
3,1491167820,nyt reporters should provide a contributors li...,22015913.0,22015913,<br/>,comment,1491150196.0,1,False,0.0,...,missouri usa,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news
4,1491167815,could only have been done in print stunning,22015466.0,22015466,<br/>,comment,1491147284.0,1,False,0.0,...,tucson arizona,nan,nan,0,58def1347c459f24986d7c80,unknown,insider,716.0,2,news


In [5]:
art_file_name = "cleaned_article_data.csv"
clean_art_csv = clean_art.to_csv(art_file_name, encoding='utf-8', index=False)

com_file_name = "cleaned_comment_data.csv"
clean_com_csv = clean_comments.to_csv(com_file_name, encoding='utf-8', index=False)